<center> <h1> Heterogeneous Computing for AI </h1> </center>

<center> <h2> Lecture 03 -: Tutorial 01 </h2> </center>

<center> <h4> Raghava Mukkamala (rrm.digi@cbs.dk)</h4> </center>

Instructions

Please use Python 3 for working on the following questions.


## Example 1 on Race Conditions

source: http://www.dabeaz.com/usenix2009/concurrent/index.html

Code adapted by Raghava

In [ ]:
# race.py
#
# A simple example of a race condition

import threading
import time

x = 0     # A shared value

COUNT = 10000000

def foo():
    global x
    for i in range(COUNT):
        x += 1

def bar():
    global x
    for i in range(COUNT):
        x -= 1


In [ ]:
start_time = time.time()

t1 = threading.Thread(target=foo)
t2 = threading.Thread(target=bar)
t1.start()
t2.start()

t1.join()
t2.join()
print ('x value:', x)

end_time = time.time()


print("Time taken : ", 
      round(end_time - start_time, 3), "Seconds")



## Example 2: Bank account to show Race Conditions

In [ ]:
'''
Standard example to display race condition & locking solution
'''

from threading import Thread
import time


# We will be using a global variable as a shared resource among the threads
shared_value = 0

def increase(amount: int) -> None:
    # allows us to use the global value instead of local (function-limited) variable 
    global shared_value

    # Copy over the shared_value to a local value
    local_val = shared_value
    # Increase local_val by the amount specified
    local_val+= amount

    # Let's make the thread sleep for half a second
    time.sleep(0.5)

    # Update shared_value 
    shared_value = local_val

    print("Value of shared_value is: ", shared_value)



In [ ]:
# reset the global value to 0.


shared_value = 0
increase(10)
increase(20)
print("Final value of shared_value is: ", shared_value)

# We would expect the solution to be 30 but due to race conditions, the solution is either 10 or 20.




In [ ]:
# reset the global value to 0.
shared_value = 0

increase(20)
increase(10)
print("Final value of shared_value is: ", shared_value)


In [ ]:
for i in range(10):
    # reset the global value to 0.
    shared_value = 0

    t1 = Thread(target= increase, args= (10,))
    t2 = Thread(target= increase, args= (20,))

    # Start Threads
    t1.start()
    t2.start()

    # Join the threads so we wait for them to complete
    t1.join()
    t2.join()

    # We would expect the solution to be 30 but due to race conditions, the solution is either 10 or 20.
    print("Final value of shared_value is: ", shared_value)




# Using Locking 

## Example 2: Bank account with Mutex lock

In [ ]:
'''
Standard example to display race condition & locking solution
'''

from threading import Thread, Lock

import time


# Create the lock

lock = Lock()

# We will be using a global variable as a shared resource among the threads
shared_value = 0

def increase(amount: int) -> None:
    # allows us to use the global value instead of local (function-limited) variable 
    global shared_value

    lock.acquire()

    # Copy over the shared_value to a local value
    local_val = shared_value
    # Increase local_val by the amount specified
    local_val+= amount

    # Let's make the thread sleep for half a second
    time.sleep(0.5)

    # Update shared_value 
    shared_value = local_val

    print("Value of shared_value is: ", shared_value)
    lock.release()



In [ ]:
for i in range(10):
    # reset the global value to 0.
    shared_value = 0

    t1 = Thread(target= increase, args= (10,))
    t2 = Thread(target= increase, args= (20,))

    # Start Threads
    t2.start()
    t1.start()


    # Join the threads so we wait for them to complete
    t1.join()
    t2.join()

    # We would expect the solution to be 30 but due to race conditions, the solution is either 10 or 20.
    print("Final value of shared_value is: ", shared_value)




## Example 1 with Mutex lock

source: http://www.dabeaz.com/usenix2009/concurrent/index.html
Code adapted by Raghava

In [ ]:
# lock_with.py
#
# A simple example of using a mutex lock for critical sections.
# Uses the context manager feature of Python 2.6.

import threading
import time

x      = 0                   # A shared value
x_lock = threading.Lock()    # A lock for synchronizing access to x

COUNT = 10000000
def foo():
    global x
    for i in range(COUNT):
        with x_lock:
            x += 1

def bar():
    global x
    for i in range(COUNT):
        with x_lock:
            x -= 1



In [ ]:
start_time = time.time()
t1 = threading.Thread(target=foo)
t2 = threading.Thread(target=bar)
t1.start()
t2.start()
t1.join()
t2.join()
print ('x value:', x)

end_time = time.time()


print("Time taken : ", 
      round(end_time - start_time, 3), "Seconds")


## Example 3:  on Semaphores

source: http://www.dabeaz.com/usenix2009/concurrent/index.html

Code adapted by Raghava

In [1]:
# sema_signal.py
#
# An example of using a semaphore to signal

import threading
import time

done = threading.Semaphore(0)
item = None

def producer():
    global item
    print ("I'm the producer and I produce data.")
    print ("Producer is going to sleep.")
    time.sleep(10)
    item = "Hello"
    print ("Producer is alive. Signaling the consumer.")
    done.release()

def consumer():
    print (threading.currentThread().name, ": I'm a consumer and I wait for data.")
    print (threading.currentThread().name, ": Consumer is waiting.")
    done.acquire()
    print (threading.currentThread().name, ": Consumer got", item)
    done.release()


In [ ]:
t1 = threading.Thread(target=producer)
t2 = threading.Thread(target=consumer)



t1.start()
t2.start()

t3 = threading.Thread(target=consumer)
t3.start()




## Example 4: on Events 

source: http://www.dabeaz.com/usenix2009/concurrent/index.html

Code adapted by Raghava

In [ ]:

# event_barrier.py
#
# An example of using an event to set up a barrier 
# synchronization

import threading
import time

init_event = threading.Event()

def worker():
    print (threading.currentThread().name, 
           " : I'm worker waiting for the event!" )
    init_event.wait()        # Wait until initialized
    print (threading.currentThread().name, 
           " : I'm worker and I am done with the event!" )


def initialize():
    print ("Initializing some data")
    time.sleep(10)
    print ("Unblocking the workers")
    init_event.set()



In [ ]:
# Launch a bunch of worker threads
threading.Thread(target=worker).start()
threading.Thread(target=worker).start()
threading.Thread(target=worker).start()
threading.Thread(target=worker).start()

# Go initialize and eventually unlock the workers
initialize()



## Example 5: on Events and Semaphores

source: http://www.dabeaz.com/usenix2009/concurrent/index.html

Code adapted by Raghava

In [11]:
# event_notify.py
#
# An example of using a semaphore and an event to have a thread
# signal completion of some task

import threading
import time

# A variable that contains some data
item = None

# A semaphore to indicate that an item is available
available_sema = threading.Semaphore(0)

# An event to indicate that processing is complete
completed_event = threading.Event()

# A worker thread
def worker():
    while True:
        available_sema.acquire()
        print (threading.currentThread().name, ": worker: processing", item)
        time.sleep(5)
        print ("worker: done")
        completed_event.set()
#         available.release()        
        

# A producer thread
def producer():
    global item
    for x in range(5):
        completed_event.clear()       # Clear the event
        item = x                # Set the item
        print ("producer: produced an item")
        available_sema.release()     # Signal on the semaphore
        completed_event.wait()
        print ("producer: item was processed")


In [12]:
start_time = time.time()

t1 = threading.Thread(target=producer)
t1.start()
t2 = threading.Thread(target=worker)
t2.setDaemon(True)
t2.start()


t1.join()

print('done with both producers and consumers!')
end_time = time.time()


print("Time taken : ", 
      round(end_time - start_time, 3), "Seconds")


producer: produced an item
Thread-33 : worker: processing 0
worker: done
producer: item was processed
producer: produced an item
Thread-33 : worker: processing 1
worker: done
producer: item was processed
producer: produced an item
Thread-33 : worker: processing 2
worker: done
producer: item was processed
producer: produced an item
Thread-33 : worker: processing 3
worker: done
producer: item was processed
producer: produced an item
Thread-33 : worker: processing 4
worker: done
producer: item was processed
done with both producers and consumers!
Time taken :  25.023 Seconds


## Example 6: on Condition Variables

source: http://www.dabeaz.com/usenix2009/concurrent/index.html

Code adapted by Raghava

In [6]:
# cond.py
#
# An example of using a condition variable to set up a producer/consumer
# problem.

import threading
import time

# A list of items that are being produced.  Note: it is actually
# more efficient to use a collections.deque() object for this.

items = []

# A condition variable for items
items_cv = threading.Condition()

# A producer thread
def producer():
    print ("I'm the producer")
    for i in range(30):
        with items_cv:          # Always must acquire the lock first
            items.append(i)     # Add an item to the list
            items_cv.notify()   # Send a notification signal
        time.sleep(1)
        
# A consumer thread
def consumer():
    print (threading.currentThread().name, ": I'm a consumer")
    while True:
        with items_cv:           # Must always acquire the lock
            while not items:     # Check if there are any items
                items_cv.wait()  # If not, we have to sleep
            x = items.pop(0)     # Pop an item off
        
        print (threading.currentThread().name," got", x)
        time.sleep(5)
    



        


In [7]:
# Launch a bunch of consumers
cons = [threading.Thread(target=consumer)
        for i in range(10)]

for c in cons:
    c.setDaemon(True)
    c.start()

# Run the producer
producer()



Thread-11 : I'm a consumer
Thread-12 : I'm a consumer
Thread-13 : I'm a consumer
Thread-14 : I'm a consumer
Thread-15 : I'm a consumer
Thread-16 : I'm a consumer
Thread-17 : I'm a consumer
Thread-18 : I'm a consumer
Thread-19 : I'm a consumer
Thread-20I'm the producer
Thread-11 : I'm a consumer
  got 0
Thread-12  got 1
Thread-13  got 2
Thread-14  got 3
Thread-15  got 4
Thread-16  got 5
Thread-17  got 6
Thread-18  got 7
Thread-19  got 8
Thread-20  got 9
Thread-11  got 10
Thread-12  got 11
Thread-13  got 12
Thread-14  got 13
Thread-15  got 14
Thread-16  got 15
Thread-17  got 16
Thread-18  got 17
Thread-19  got 18
Thread-20  got 19
Thread-11  got 20
Thread-12  got 21
Thread-13  got 22
Thread-14  got 23
Thread-15  got 24
Thread-16  got 25
Thread-17  got 26
Thread-18  got 27
Thread-19  got 28
Thread-20  got 29


## Example on Ques

source: http://www.dabeaz.com/usenix2009/concurrent/index.html

Code adapted by Raghava

In [8]:
# pc_queue.py
#
# An example of using queues to set up producer/consumer problems

import threading
import time
import queue

# A queue of items being produced

items = queue.Queue()

# A producer thread
def producer():
    print ("I'm the producer")
    for i in range(30):
        items.put(i)
        time.sleep(1)
        
# A consumer thread
def consumer():
    print (threading.currentThread().name, ": I'm a consumer")
    while True:
        x = items.get() 
        print (threading.currentThread().name,": got", x)
        time.sleep(5)
    
      


In [9]:
# Launch a bunch of consumers
cons = [threading.Thread(target=consumer)
        for i in range(10)]
for c in cons:
    c.setDaemon(True)
    c.start()

# Run the producer
producer()


Thread-21 : I'm a consumer
Thread-22 : I'm a consumer
Thread-23 : I'm a consumer
Thread-24 : I'm a consumer
Thread-25 Thread-26: I'm a consumer
Thread-27  : I'm a consumer
: I'm a consumer
Thread-28 : I'm a consumer
Thread-29 : I'm a consumer
Thread-30 : I'm a consumer
I'm the producer
Thread-21 : got 0
Thread-22 : got 1
Thread-23 : got 2
Thread-24 : got 3
Thread-25 : got 4
Thread-27 : got 5
Thread-26 : got 6
Thread-28 : got 7
Thread-29 : got 8
Thread-30 : got 9
Thread-21 : got 10
Thread-22 : got 11
Thread-23 : got 12
Thread-24 : got 13
Thread-25 : got 14
Thread-27 : got 15
Thread-26 : got 16
Thread-28 : got 17
Thread-29 : got 18
Thread-30 : got 19
Thread-21 : got 20
Thread-22 : got 21
Thread-23 : got 22
Thread-24 : got 23
Thread-25 : got 24
Thread-27 : got 25
Thread-26 : got 26
Thread-28 : got 27
Thread-29 : got 28
Thread-30 : got 29


## Another example on Queues 

(also to be used in exercise)

In [10]:
import threading
import queue

q = queue.Queue()

def worker():
    while True:
        print("In Iteration ")
        item = q.get()
        print(f'Working on {item}')
        print(f'Finished {item}')
        q.task_done()

# Turn-on the worker thread.
threading.Thread(target=worker, daemon=True).start()

# Send thirty task requests to the worker.
for item in range(10):
    q.put(item)

# Block until all tasks are done.
q.join()
print('All work completed')


# Exercise: Utilsing a Queue.

# Use the old exercise from week 2 with the downloading of url and computing length.

# Create a producer/consumer threads.

# One thread will download the file and put the contents in a queue

# second thread will compute the length of the contents maybe?? 
# That could work!




In Iteration 
Working on 0
Finished 0
In Iteration 
Working on 1
Finished 1
In Iteration 
Working on 2
Finished 2
In Iteration 
Working on 3
Finished 3
In Iteration 
Working on 4
Finished 4
In Iteration 
Working on 5
Finished 5
In Iteration 
Working on 6
Finished 6
In Iteration 
Working on 7
Finished 7
In Iteration 
Working on 8
Finished 8
In Iteration 
Working on 9
Finished 9
In Iteration 
All work completed
